In [15]:
import os
import pandas as pd
import numpy as np
import h5py

# load data
df_train=pd.read_csv('dataset/train_label.csv')
df_test=pd.read_csv('dataset/val_label.csv')

# split train/val
sp=pd.read_csv('dataset/val_id.csv',header=None).values
sp=np.unique(np.array(list(map(lambda x:x[0].split('_')[0],sp))))
train_index=[]
val_index=[]
videos=df_train['video'].values
for i,v in enumerate(videos):
    if v.split('_')[0] in sp:
        val_index.append(i)
    else:
        train_index.append(i)
df_val=df_train.iloc[val_index,:]
df_train=df_train.iloc[train_index,:]
df_train.head()

id_train = df_train[['video','utterance']].values
id_train = np.array(list(map(lambda x:x[0].split('_')[0]+'_'+x[1].split('.')[0].split('_')[-1],id_train)))
id_val = df_val[['video','utterance']].values
id_val = np.array(list(map(lambda x:x[0].split('_')[0]+'_'+x[1].split('.')[0].split('_')[-1],id_val)))
id_test = df_test[['video','utterance']].values
id_test = np.array(list(map(lambda x:x[0].split('_')[0]+'_'+x[1].split('.')[0].split('_')[-1],id_test)))

# remove_list
remove_list=['cbb64e001_14','9354b482f_9','2c09795bb_40']
remove_index=[]
for i,id in enumerate(id_train):
    if id in remove_list:
        print('remove',id)
        remove_index.append(i)
remove_index=np.array(remove_index)
print(remove_index)
print(id_train.shape)
id_train=np.delete(id_train, remove_index, axis=0)

y_train=df_train[['arousal','valence']].values
y_train=np.delete(y_train,remove_index, axis=0)
y_val=df_val[['arousal','valence']].values
y_test=df_test[['arousal','valence']].values

from tqdm import tqdm
# read vision feature
# train
data = h5py.File('/users/seria/TensorFlood/data/seeta_omg/vf-full-train.hdf5', 'r')# vid_vfssdaw_train
X_train_vid=[]
videos=df_train['video'].values
utts=df_train['utterance'].values
for i in tqdm(range(len(videos))):
    if (videos[i].split('_')[0]+'_'+utts[i].split('.')[0].split('_')[-1]) not in remove_list:
        x_temp=data[videos[i]+'_'+utts[i].split('.')[0].split('_')[-1]][:]
        X_train_vid.append(x_temp)
X_train_vid=np.array(X_train_vid)
print(X_train_vid.shape)

# val
data = h5py.File('/users/seria/TensorFlood/data/seeta_omg/vf-full-val.hdf5', 'r') # vid_vfssdaw_val
X_val_vid=[]
videos=df_val['video'].values
utts=df_val['utterance'].values
for i in tqdm(range(len(videos))):
    x_temp=data[videos[i]+'_'+utts[i].split('.')[0].split('_')[-1]][:]
    X_val_vid.append(x_temp)
X_val_vid=np.array(X_val_vid)
print(X_val_vid.shape)

# test
data = h5py.File('/users/seria/TensorFlood/data/seeta_omg/vf-full-test.hdf5', 'r') # vid_vfssdaw_test
X_test_vid=[]
videos=df_test['video'].values
utts=df_test['utterance'].values
for i in tqdm(range(len(videos))):
    x_temp=data[videos[i]+'_'+utts[i].split('.')[0].split('_')[-1]][:]
    X_test_vid.append(x_temp)
X_test_vid=np.array(X_test_vid)
print(X_test_vid.shape)

 13%|█▎        | 250/1972 [00:00<00:00, 2499.21it/s]

remove cbb64e001_14
remove 2c09795bb_40
remove 9354b482f_9
[119 197 563]
(1972,)


 44%|████▍     | 206/470 [00:00<00:00, 2057.15it/s]

(1969, 256)


 33%|███▎      | 204/617 [00:00<00:00, 1897.10it/s]

(470, 256)


100%|██████████| 617/617 [00:00<00:00, 2148.70it/s]

(617, 256)


In [25]:
import pandas as pd
import numpy as np

paths={'aud_CNN':['aud_feature/AudCNN_context0_train_feat.csv',\
                  'aud_feature/AudCNN_context0_val_feat.csv',\
                  'aud_feature/AudCNN_context0_test_feat.csv'],\
      'aud_hand':['../OMG/dataset/aud_train.csv',\
                 '../OMG/dataset/aud_val.csv'],\
      'text_attention':['text_feature/TextAttention_context0_train_feat256.csv',\
                       'text_feature/TextAttention_context0_val_feat256.csv',\
                       'text_feature/TextAttention_context0_test_feat256.csv'],\
      'text_BiGRU':['text_feature/TextGRU_context0_train_feat256.csv',\
                   'text_feature/TextGRU_context0_val_feat256.csv',\
                   'text_feature/TextGRU_context0_test_feat256.csv'],
      'vid_CNN':['vid_feature/vidCNN_context0_train_feat.csv',\
                'vid_feature/vidCNN_context0_val_feat.csv',\
                'vid_feature/vidCNN_context0_test_feat.csv']}
# 选择非video特征
# aud_CNN
# aud_hand
# text_attention
# text_BiGRU
feature_list=['aud_hand','text_attention']
from sklearn.preprocessing import normalize
def combine_features(feature_list, remove_index, id_train, id_val, id_test):
    X_train_all=[]
    X_val_all=[]
    X_test_all=[]
    for f in feature_list:
        if f=='aud_hand':
            df_train=pd.read_csv(paths[f][0])
            df_val=pd.read_csv(paths[f][1])
            id_train2=df_train.pop('id').values
            id_val2=df_val.pop('id').values
            df_train.pop('y1');df_val.pop('y1');df_train.pop('y2');df_val.pop('y2')
            X_train_temp=df_train.values
            X_val_temp=df_val.values
            train_index_dict={};val_index_dict={}
            X_train_index=[];X_val_index=[];X_test_index=[]
            for i,id in enumerate(id_train2):
                train_index_dict[id]=i
            for i,id in enumerate(id_val2):
                val_index_dict[id]=i
            for id in id_train:
                X_train_index.append(train_index_dict[id])
            for id in id_val:
                X_val_index.append(train_index_dict[id])
            for id in id_test:
                X_test_index.append(val_index_dict[id])
            X_train_index=np.array(X_train_index);X_val_index=np.array(X_val_index);X_test_index=np.array(X_test_index)
            X_test_temp=X_val_temp[X_test_index,:]
            X_val_temp=X_train_temp[X_val_index,:]
            X_train_temp=X_train_temp[X_train_index,:]
        else:
            #load data
            df_train=pd.read_csv(paths[f][0])
            df_val=pd.read_csv(paths[f][1])
            df_test=pd.read_csv(paths[f][2])
            X_train_temp=df_train.iloc[:,4:].values
            X_val_temp=df_val.iloc[:,4:].values
            X_test_temp=df_test.iloc[:,4:].values
            if f!='vid_CNN':
                X_train_temp=np.delete(X_train_temp, remove_index, axis=0)
#             if f=='aud_CNN':
#                 # l2 norm
#                 X_train_temp=normalize(X_train_temp)
#                 X_val_temp=normalize(X_val_temp)
#                 X_test_temp=normalize(X_test_temp)
        X_train_all.append(X_train_temp)
        X_val_all.append(X_val_temp)
        X_test_all.append(X_test_temp)
    X_train_all=np.concatenate(X_train_all,axis=1)
    X_val_all=np.concatenate(X_val_all,axis=1)
    X_test_all=np.concatenate(X_test_all,axis=1)
    
    return X_train_all,X_val_all,X_test_all

X_train,X_val,X_test=combine_features(feature_list, remove_index, id_train, id_val, id_test)
print(X_train.shape,X_val.shape,X_test.shape)
# concatenate
X_train=np.concatenate((X_train,X_train_vid),axis=1)
X_val=np.concatenate((X_val,X_val_vid),axis=1)
X_test=np.concatenate((X_test,X_test_vid),axis=1)

# X_train=X_train_vid
# X_val=X_val_vid
# X_test=X_test_vid

# # min-max normalization
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# X_all=np.concatenate((X_train,X_val,X_test),axis=0)
# scaler.fit(X_all)
# X_train=scaler.transform(X_train)
# X_val=scaler.transform(X_val)
# X_test=scaler.transform(X_test)
print('concated and normed:')
print(X_train.shape,X_val.shape,X_test.shape)

(1969, 512) (470, 512) (617, 512)
concated and normed:
(1969, 768) (470, 768) (617, 768)


In [26]:
# a
def correct(train_y, pred_val_y):
    train_std = np.std(train_y)
    val_std = np.std(pred_val_y)
    mean = np.mean(pred_val_y)
    pred_val_y = np.array(pred_val_y)
    pred_val_y = mean + (pred_val_y - mean) * train_std / val_std
    return pred_val_y

import calccc
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import cross_val_score
from sklearn import svm
import numpy as np
params = {
        'gamma': ['auto'],
        'C': [0.01,0.05,0.1,0.5,1,5,10,50,100,200],#
        'kernel':['rbf']
    }
params = list(ParameterGrid(params))
best_ccc=0
for p in params:
    print(p)
    clf = svm.SVR(kernel=p['kernel'], C=p['C'], gamma=p['gamma'])#
    clf.fit(X_train,y_train[:,0])
    y_pred = clf.predict(X_val)
    y_pred2 = correct(y_train[:,0], y_pred)
    ccc,_=calccc.ccc(y_val[:,0],y_pred)
    ccc2,_=calccc.ccc(y_val[:,0],y_pred2)
    print('ccc:',ccc,'(',ccc2,')')
    if ccc>best_ccc:
        best_ccc=ccc
        best_param=p
        best_clf=clf
        
print('best_ccc:',best_ccc)
print('best_param!!!:',best_param)

y_pred = best_clf.predict(X_test)
y_pred2 = correct(y_train[:,0], y_pred)
ccc1,_=calccc.ccc(y_test[:,0],y_pred)
ccc2,_=calccc.ccc(y_test[:,0],y_pred2)
print('test ccc:',ccc1,'(',ccc2,')')

{'C': 0.01, 'gamma': 'auto', 'kernel': 'rbf'}
ccc: 0.164842402889 ( 0.382825038176 )
{'C': 0.05, 'gamma': 'auto', 'kernel': 'rbf'}
ccc: 0.24674233729 ( 0.392395465634 )
{'C': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}
ccc: 0.260806894109 ( 0.382050362312 )
{'C': 0.5, 'gamma': 'auto', 'kernel': 'rbf'}
ccc: 0.25824372087 ( 0.339734127662 )
{'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}
ccc: 0.248284133357 ( 0.316211714189 )
{'C': 5, 'gamma': 'auto', 'kernel': 'rbf'}
ccc: 0.229863039545 ( 0.282727089181 )
{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
ccc: 0.22467281918 ( 0.2745459775 )
{'C': 50, 'gamma': 'auto', 'kernel': 'rbf'}
ccc: 0.220176109595 ( 0.268961711218 )
{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}
ccc: 0.220176109595 ( 0.268961711218 )
{'C': 200, 'gamma': 'auto', 'kernel': 'rbf'}
ccc: 0.220176109595 ( 0.268961711218 )
best_ccc: 0.260806894109
best_param!!!: {'C': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}
test ccc: 0.213837379197 ( 0.301881825666 )


In [27]:
params = {
        'gamma': ['auto'],
        'C': [0.01,0.05,0.1,0.5,1,5,10,50,100,200],#
        'kernel':['rbf']
    }
params = list(ParameterGrid(params))
best_ccc=0
for p in params:
    print(p)
    clf = svm.SVR(kernel=p['kernel'], C=p['C'], gamma=p['gamma'])#
    clf.fit(X_train,y_train[:,1])
    y_pred = clf.predict(X_val)
    y_pred2 = correct(y_train[:,1], y_pred)
    ccc,_=calccc.ccc(y_val[:,1],y_pred)
    ccc2,_=calccc.ccc(y_val[:,1],y_pred2)
    print('ccc:',ccc,'(',ccc2,')')
    if ccc>best_ccc:
        best_ccc=ccc
        best_param=p
        best_clf=clf
        
print('best_ccc:',best_ccc)
print('best_param!!!:',best_param)

y_pred = best_clf.predict(X_test)
y_pred2 = correct(y_train[:,1], y_pred)
ccc1,_=calccc.ccc(y_test[:,1],y_pred)
ccc2,_=calccc.ccc(y_test[:,1],y_pred2)
print('test ccc:',ccc1,'(',ccc2,')')

{'C': 0.01, 'gamma': 'auto', 'kernel': 'rbf'}
ccc: 0.299731002991 ( 0.512887992407 )
{'C': 0.05, 'gamma': 'auto', 'kernel': 'rbf'}
ccc: 0.407437616958 ( 0.516745990482 )
{'C': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}
ccc: 0.433966089496 ( 0.516087912788 )
{'C': 0.5, 'gamma': 'auto', 'kernel': 'rbf'}
ccc: 0.48143715699 ( 0.523255660701 )
{'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}
ccc: 0.479340072111 ( 0.515587173145 )
{'C': 5, 'gamma': 'auto', 'kernel': 'rbf'}
ccc: 0.469782546635 ( 0.499084433338 )
{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
ccc: 0.465777316616 ( 0.493068068177 )
{'C': 50, 'gamma': 'auto', 'kernel': 'rbf'}
ccc: 0.463641183104 ( 0.48970169329 )
{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}
ccc: 0.465323122246 ( 0.490386003516 )
{'C': 200, 'gamma': 'auto', 'kernel': 'rbf'}
ccc: 0.465323122246 ( 0.490386003516 )
best_ccc: 0.48143715699
best_param!!!: {'C': 0.5, 'gamma': 'auto', 'kernel': 'rbf'}
test ccc: 0.239419071175 ( 0.275093753331 )


In [5]:
y_pred_1=y_pred

In [14]:
y_pred_2=y_pred
y_pred_fin=0.6*y_pred_1+0.4*y_pred_2
y_pred_fin2=correct(y_train[:,1],y_pred_fin)
ccc1,_=calccc.ccc(y_test[:,1],y_pred_fin)
ccc2,_=calccc.ccc(y_test[:,1],y_pred_fin2)
print('test ccc:',ccc1,'(',ccc2,')')

test ccc: 0.469285291829 ( 0.477291911208 )


In [ ]:
# a
def correct(train_y, pred_val_y):
    train_std = np.std(train_y)
    val_std = np.std(pred_val_y)
    mean = np.mean(pred_val_y)
    pred_val_y = np.array(pred_val_y)
    pred_val_y = mean + (pred_val_y - mean) * train_std / val_std
    return pred_val_y

# 自定义ccc评价
def metric_ccc(preds,lgbdata):
    labels=lgbdata.get_label() 
    ccc,_=calccc.ccc(labels,preds)
    return 'ccc value:',ccc,True

# lgb
import calccc
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
import numpy as np
params = {
    'metric': ['metric_ccc'],
    'application': ['regression'],
    'learning_rate':[0.015],
    'feature_fraction': [0.95],
    'max_depth': [6],
    'num_leaves':[50],
    'bagging_fraction': [0.95],
    'bagging_freq':[5],
    'min_data_in_leaf':[10],
    'min_gain_to_split':[0],
    'num_iterations':[1000],
    'lambda_l1':[0.5],
    'lambda_l2':[1.5],
    'verbose':[1]
}
params=list(ParameterGrid(params))
lgbtrain=lgb.Dataset(X_train,label=y_train[:,0])
lgbeval=lgb.Dataset(X_val,label=y_val[:,0],reference=lgbtrain)
best_ccc=0
for param in params:
    print(param)
    clf = lgb.train(param, lgbtrain, valid_sets=lgbeval, num_boost_round=param['num_iterations'], \
                    early_stopping_rounds=50, feval=metric_ccc, verbose_eval=True)
    print(clf.best_score)
    if clf.best_score['valid_0']['ccc value:']>best_ccc:
        best_ccc=clf.best_score['valid_0']['ccc value:']
        best_param=param
        best_it=clf.best_iteration
        best_clf=clf
    print('noval best interation: '+str(clf.best_iteration))
y_pred = clf.predict(X_val)
y_pred2 = correct(y_train[:,0], y_pred)
ccc2,_=calccc.ccc(y_val[:,0],y_pred2)
print('best validation ccc:',best_ccc,'(',ccc2,')')
print('best param:',best_param)
print('best iteration:',best_it)

y_pred = best_clf.predict(X_test)
y_pred2 = correct(y_train[:,0], y_pred)
ccc1,_=calccc.ccc(y_test[:,0],y_pred)
ccc2,_=calccc.ccc(y_test[:,0],y_pred2)
print('test ccc:',ccc1,'(',ccc2,')')